## Reqs

In [ ]:
!pip install -U scikit-learn==1.0.1
!git install -U tensorflow==2.7.0
!pip install -U seaborn==0.11.2
!pip install -U matplotlib==3.5.0
!pip install -U pandas==1.3.4
%cd DataSets

In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics
import tensorflow as tf

import warnings
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Read class and package metrics(x:inputs) and RR(y:label)

In [18]:
cls_df = pd.read_csv('cls_normalized.csv')
pkg_df = pd.read_csv('pkg_normalized.csv')
cls_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pkg_df.drop(['Unnamed: 0'], axis=1, inplace=True)

## Class DNN

### Load data

In [33]:
#target_scaler = MinMaxScaler(feature_range=(-10, 10))
data = cls_df
df = data.iloc[:,3:-2]
data_columns = df.columns
X_np = np.array(df)
#target_scaler.fit(X_np)
X = X_np
#X = target_scaler.transform(X_np)

Y = data.iloc[:,-1]
Y = Y.values.reshape(-1, 1)
#target_scaler = MinMaxScaler(feature_range=(-10, 10))
#target_scaler.fit(Y)
#Y = target_scaler.transform(Y)

In [38]:
len(X[0])

27

### Training data preparation 

In [51]:
#indexes = np.arange(len(X))
splitter = int(len(X) * .8)
X_train = X[:splitter].astype('float32')
Y_train = Y[:splitter].astype('int32')
X_val = X[splitter:].astype('float32')
Y_val = Y[splitter:].astype('int32')

In [79]:
X_val

array([[-0.9881657 , -0.75      , -0.4       , ..., -0.98737085,
        -0.9793256 , -0.99450547],
       [-0.9881657 , -0.75      , -0.4       , ..., -0.9965557 ,
        -0.9906473 , -0.9958791 ],
       [-0.9881657 , -0.9166667 , -0.8       , ..., -0.9690012 ,
        -0.98031014, -0.9684066 ],
       ...,
       [-0.9881657 , -0.9166667 , -0.8       , ..., -0.9988519 ,
        -0.99753875, -0.99862635],
       [-0.9881657 , -1.        , -1.        , ..., -0.9988519 ,
        -0.9995077 , -0.99862635],
       [-0.9881657 , -1.        , -1.        , ..., -0.9988519 ,
        -0.9995077 , -0.99862635]], dtype=float32)

### Train model

In [102]:
##model


def my_model_fully_connected(loss_fn, opt, ishape = 27, lr = 1e-2):
    model_input = tf.keras.layers.Input(shape = ishape)
    l2 = tf.keras.layers.Dense(50, activation = 'sigmoid')(model_input)
    l2_drop = tf.keras.layers.Dropout(0.02)(l2)
    l3 = tf.keras.layers.Dense(25, activation = 'sigmoid')(l2_drop)
    out= tf.keras.layers.Dense(1, activation = 'softmax')(l3)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=opt, loss=loss_fn, metrics = [tf.keras.metrics.CategoricalAccuracy()])
    return model

lr = 0.001
opt = 'sgd'
loss_fn = 'mse'
model = my_model_fully_connected(loss_fn, opt, ishape=len(X_train[0]), lr= lr)
print(model.summary())


Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 27)]              0         
                                                                 
 dense_83 (Dense)            (None, 50)                1400      
                                                                 
 dropout_23 (Dropout)        (None, 50)                0         
                                                                 
 dense_84 (Dense)            (None, 25)                1275      
                                                                 
 dense_85 (Dense)            (None, 1)                 26        
                                                                 
Total params: 2,701
Trainable params: 2,701
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
Y_train[0:20]

array([[4],
       [3],
       [4],
       [4],
       [4],
       [4],
       [4],
       [0],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [4],
       [0],
       [4],
       [0],
       [0],
       [0]], dtype=int32)

In [104]:
# Call backs 
model_name_fcn = "First_draft.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_fcn, monitor='val_loss',\
                                             verbose=0, save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 0)

In [105]:
# Train
model.fit(X_train, Y_train, epochs = 100, batch_size=200, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule], validation_data= (X_val, Y_val))

Epoch 1/100
100/100 [==============================] - 1s 6ms/step - loss: 1.7906 - categorical_accuracy: 1.0000 - val_loss: 1.8274 - val_categorical_accuracy: 1.0000 - lr: 0.0050
Epoch 2/100
100/100 [==============================] - 0s 4ms/step - loss: 1.7906 - categorical_accuracy: 1.0000 - val_loss: 1.8274 - val_categorical_accuracy: 1.0000 - lr: 0.0050
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 1.7906 - categorical_accuracy: 1.0000 - val_loss: 1.8274 - val_categorical_accuracy: 1.0000 - lr: 0.0050
Epoch 4/100
100/100 [==============================] - 0s 4ms/step - loss: 1.7906 - categorical_accuracy: 1.0000 - val_loss: 1.8274 - val_categorical_accuracy: 1.0000 - lr: 0.0050
Epoch 5/100
100/100 [==============================] - 1s 5ms/step - loss: 1.7906 - categorical_accuracy: 1.0000 - val_loss: 1.8274 - val_categorical_accuracy: 1.0000 - lr: 0.0050
Epoch 6/100
100/100 [==============================] - 0s 4ms/step - loss: 1.7906 - categorical_accu

In [106]:

model.predict(X_val)

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

4

## Package DNN

### Load data

In [ ]:
target_scaler = MinMaxScaler()
data = pkg_df
df = data.iloc[:,3:-1]
data_columns = df.columns
X_np = np.array(df)
target_scaler.fit(X_np)
X = target_scaler.transform(X_np)

Y = data.iloc[:,-1]
Y = Y.values.reshape(-1, 1)
target_scaler = MinMaxScaler()
target_scaler.fit(Y)
Y = target_scaler.transform(Y)


In [ ]:
splitter = int(len(X) * .8)
X_train = X[:splitter]
Y_train = Y[:splitter]
X_val = X[splitter:]
Y_val = Y[splitter:]

### Train model

In [ ]:
def my_model_fully_connected(loss_fn, opt, ishape = 27, lr = 1e-2):
    model_input = tf.keras.layers.Input(shape = ishape)
    l2 = tf.keras.layers.Dense(100, activation = 'sigmoid')(model_input)
    l2_drop = tf.keras.layers.Dropout(0.05)(l2)
    l3 = tf.keras.layers.Dense(50, activation = 'relu')(l2_drop)
    l4 = tf.keras.layers.Dense(25, activation = 'relu')(l3) 
    out= tf.keras.layers.Dense(1, activation = 'relu')(l4)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=opt, loss=loss_fn, metrics = ["accuracy"])
    return model

lr = 0.001
opt = opt = tf.keras.optimizers.SGD(learning_rate=lr)
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError()
model = my_model_fully_connected(loss_fn, opt, ishape=len(X_train[0]), lr= lr)
print(model.summary())


Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 18)]              0         
                                                                 
 dense_33 (Dense)            (None, 100)               1900      
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_34 (Dense)            (None, 50)                5050      
                                                                 
 dense_35 (Dense)            (None, 25)                1275      
                                                                 
 dense_36 (Dense)            (None, 1)                 26        
                                                                 
Total params: 8,251
Trainable params: 8,251
Non-trainable 

In [ ]:
model_name_fcn = "package_draft.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 200)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_fcn, monitor='val_loss',\
                                             verbose=0, save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 0)

In [ ]:
## Train model
model.fit(X_train, Y_train, epochs = 100, batch_size=32, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule], validation_data= (X_val, Y_val))

Epoch 1/100
55/55 [==============================] - 1s 12ms/step - loss: 0.0238 - accuracy: 0.5251 - val_loss: 0.0109 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 2/100
55/55 [==============================] - 0s 5ms/step - loss: 0.0142 - accuracy: 0.5251 - val_loss: 0.0051 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 3/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0084 - accuracy: 0.5251 - val_loss: 0.0027 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 4/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0055 - accuracy: 0.5251 - val_loss: 0.0019 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 5/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0035 - accuracy: 0.5251 - val_loss: 0.0017 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 6/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 0.5251 - val_loss: 0.0017 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 7/100
55/55 [==============================] 

KeyboardInterrupt: ignored

In [ ]:
## Test model